In [1]:
import pandas as pd
import numpy as np
import os
import cv2
import subprocess
import imageio
import imageio_ffmpeg as ffmpeg
import pickle
from IPython.display import Video, display, Image

In [2]:
def convert_video_to_h264_via_avi(input_path, temp_avi="temp.avi", output_path="preview_h264.mp4"):
    # 读取输入视频
    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        raise IOError(f"无法打开视频文件: {input_path}")
    
    # 获取视频属性
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    # 以 AVI 格式保存临时视频，使用 MJPG 编码
    fourcc = cv2.VideoWriter_fourcc(*'MJPG')
    avi_writer = cv2.VideoWriter(temp_avi, fourcc, fps, (width, height))
    if not avi_writer.isOpened():
        raise IOError(f"无法创建 VideoWriter，请检查编码器或路径: {temp_avi}")
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        avi_writer.write(frame)
    
    cap.release()
    avi_writer.release()
    
    # 使用 imageio-ffmpeg 提供的 ffmpeg 二进制文件进行转码
    ffmpeg_exe = ffmpeg.get_ffmpeg_exe()
    cmd = [
        ffmpeg_exe, "-y", "-i", temp_avi,
        "-c:v", "libx264", "-preset", "fast",
        output_path
    ]
    subprocess.run(cmd, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    
    # 在 Notebook 中显示生成的 H.264 视频
    display(Video(output_path, embed=True))

In [48]:

dir_name = "rD_baseline"
dir_wrong_data_info = "/nfs/turbo/coe-mcity/hanhy/LNDE_Results/" + dir_name + "/check/"
dir_scene_videos = "/nfs/turbo/coe-mcity/hanhy/LNDE_Results/" + dir_name + "/scene_videos/"

file_dir = "8632"

data = pickle.load(open(dir_wrong_data_info + file_dir + ".pkl", "rb"))
wrong_info = data["wrong_and_sigma"] #(time, 7)

wrong_time_1 = np.argwhere(wrong_info[:, :, 0] > 0.5)
wrong_time_2 = np.argwhere(wrong_info[:, :, 1] > 0.5)
wrong_time_3 = len(np.argwhere(wrong_info[:, :, 2] > 0.5)) > 0
# wrongpred3 = np.argwhere(wrong_sigma_info3[:, :, 3] > 0.5)
# wrongpred4 = np.argwhere(wrong_sigma_info3[:, :, 4] > 0.5)

print(f"video_len:{wrong_info.shape[1] + 1}")
print("wrong dis")
print(wrong_time_1)
print('wrong angle:')
print(wrong_time_2)
print('if small poc:')
print(wrong_time_3)

# print("----------------")
# print(wrongpred3)
# print("----------------")
# print(wrongpred4)
#show video
save_preview_dir = "/home/hanhy/ondemand/data/sys/myjobs/Conflict_Identifier_Network/priview_output_video/"
video_name = dir_scene_videos + file_dir + ".mp4"
assert os.path.exists(video_name)
convert_video_to_h264_via_avi(input_path=video_name, temp_avi=save_preview_dir + "temp.avi", output_path=save_preview_dir + "preview_h264.mp4")

video_len:42
wrong dis
[[ 6 40]]
wrong angle:
[[ 4 39]
 [ 6 39]
 [ 6 40]]
if small poc:
False
